# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

For all the answers, the source of information is : https://arxiv.org/pdf/1411.4555.pdf

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:** The model is comprised of a CNN that computes a low level representation of the images that are passed through it and in turn, this representation is fed into an LSTM cell that is in charge of producing the corresponding natural language description. 


### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** The parameters in Resnet where trained by passing through it a series of images that were transformed in a specific way. The provided transformations match the latter. 

### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:** By looking at the paper I reference above(https://arxiv.org/pdf/1411.4555.pdf), the authors expressed the following regarding the matter:

"The above loss (referring to NLLLoss ) is minimized w.r.t. all the parameters of theLSTM, the top layer of the image embedder CNN and word embeddings We."

### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:** Following with the reference in Question 3, the authors used stochastic gradient descent. I quote: "At training time, (S, I) is a training example pair, and we optimize the sum of the log probabilities as described in (2) over the whole training set using stochastic gradient descent". By trial and error, I have found that Adam works better, however.

In [2]:
#performing imports first
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math
import nltk

In [3]:
## TODO #1: Select appropriate values for the Python variables below.
batch_size = 64          # batch size
vocab_threshold = 5        # minimum word count threshold
vocab_from_file = True    # if True, load existing vocab file
embed_size = 256           # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder
num_epochs = 3             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...
Done (t=0.89s)
creating index...


  0%|          | 784/414113 [00:00<01:52, 3679.35it/s]

index created!
Obtaining caption lengths...


100%|██████████| 414113/414113 [01:32<00:00, 4469.46it/s]


[Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False), BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), ReLU(inplace), MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False), Sequential(
  (0): Bottleneck(
    (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (downsample): Sequential(
      (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_

In [4]:
# TODO #3: Specify the learnable parameters of the model.
params = list(encoder.embed.parameters()) + list(decoder.parameters())

# TODO #4: Define the optimizer.
optimizer = torch.optim.Adam(params, lr=0.001)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [ ]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

# Open the training log file.
f = open(log_file, 'w')

old_time = time.time()
response = requests.request("GET", 
                            "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
                            headers={"Metadata-Flavor":"Google"})

for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        if time.time() - old_time > 60:
            old_time = time.time()
            requests.request("POST", 
                             "https://nebula.udacity.com/api/v1/remote/keep-alive", 
                             headers={'Authorization': "STAR " + response.text})
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))

# Close the training log file.
f.close()

Shape of captions
torch.Size([64, 10, 256])
Shape of inputs
torch.Size([64, 11, 256])
Epoch [1/3], Step [1/6471], Loss: 9.0905, Perplexity: 8870.8137Shape of captions
torch.Size([64, 14, 256])
Shape of inputs
torch.Size([64, 15, 256])
Epoch [1/3], Step [2/6471], Loss: 8.9630, Perplexity: 7808.6541Shape of captions
torch.Size([64, 13, 256])
Shape of inputs
torch.Size([64, 14, 256])
Epoch [1/3], Step [3/6471], Loss: 8.7759, Perplexity: 6476.2445Shape of captions
torch.Size([64, 14, 256])
Shape of inputs
torch.Size([64, 15, 256])
Epoch [1/3], Step [4/6471], Loss: 8.5691, Perplexity: 5266.4452Shape of captions
torch.Size([64, 10, 256])
Shape of inputs
torch.Size([64, 11, 256])
Epoch [1/3], Step [5/6471], Loss: 8.1110, Perplexity: 3331.0091Shape of captions
torch.Size([64, 19, 256])
Shape of inputs
torch.Size([64, 20, 256])
Epoch [1/3], Step [6/6471], Loss: 7.9977, Perplexity: 2974.2039Shape of captions
torch.Size([64, 14, 256])
Shape of inputs
torch.Size([64, 15, 256])
Epoch [1/3], Step [7

Epoch [1/3], Step [56/6471], Loss: 4.0546, Perplexity: 57.6594Shape of captions
torch.Size([64, 10, 256])
Shape of inputs
torch.Size([64, 11, 256])
Epoch [1/3], Step [57/6471], Loss: 4.2397, Perplexity: 69.3860Shape of captions
torch.Size([64, 14, 256])
Shape of inputs
torch.Size([64, 15, 256])
Epoch [1/3], Step [58/6471], Loss: 4.3304, Perplexity: 75.9741Shape of captions
torch.Size([64, 10, 256])
Shape of inputs
torch.Size([64, 11, 256])
Epoch [1/3], Step [59/6471], Loss: 4.3320, Perplexity: 76.0925Shape of captions
torch.Size([64, 10, 256])
Shape of inputs
torch.Size([64, 11, 256])
Epoch [1/3], Step [60/6471], Loss: 4.1013, Perplexity: 60.4159Shape of captions
torch.Size([64, 17, 256])
Shape of inputs
torch.Size([64, 18, 256])
Epoch [1/3], Step [61/6471], Loss: 4.4930, Perplexity: 89.3931Shape of captions
torch.Size([64, 9, 256])
Shape of inputs
torch.Size([64, 10, 256])
Epoch [1/3], Step [62/6471], Loss: 4.2991, Perplexity: 73.6324Shape of captions
torch.Size([64, 12, 256])
Shape o

Epoch [1/3], Step [111/6471], Loss: 3.7258, Perplexity: 41.5053Shape of captions
torch.Size([64, 14, 256])
Shape of inputs
torch.Size([64, 15, 256])
Epoch [1/3], Step [112/6471], Loss: 3.8520, Perplexity: 47.0887Shape of captions
torch.Size([64, 10, 256])
Shape of inputs
torch.Size([64, 11, 256])
Epoch [1/3], Step [113/6471], Loss: 3.8536, Perplexity: 47.1614Shape of captions
torch.Size([64, 19, 256])
Shape of inputs
torch.Size([64, 20, 256])
Epoch [1/3], Step [114/6471], Loss: 4.2621, Perplexity: 70.9566Shape of captions
torch.Size([64, 13, 256])
Shape of inputs
torch.Size([64, 14, 256])
Epoch [1/3], Step [115/6471], Loss: 3.8783, Perplexity: 48.3398Shape of captions
torch.Size([64, 17, 256])
Shape of inputs
torch.Size([64, 18, 256])
Epoch [1/3], Step [116/6471], Loss: 4.2102, Perplexity: 67.3712Shape of captions
torch.Size([64, 11, 256])
Shape of inputs
torch.Size([64, 12, 256])
Epoch [1/3], Step [117/6471], Loss: 3.7933, Perplexity: 44.4008Shape of captions
torch.Size([64, 11, 256])

Epoch [1/3], Step [166/6471], Loss: 3.5233, Perplexity: 33.8966Shape of captions
torch.Size([64, 19, 256])
Shape of inputs
torch.Size([64, 20, 256])
Epoch [1/3], Step [167/6471], Loss: 4.2125, Perplexity: 67.5222Shape of captions
torch.Size([64, 14, 256])
Shape of inputs
torch.Size([64, 15, 256])
Epoch [1/3], Step [168/6471], Loss: 3.7127, Perplexity: 40.9651Shape of captions
torch.Size([64, 10, 256])
Shape of inputs
torch.Size([64, 11, 256])
Epoch [1/3], Step [169/6471], Loss: 3.7794, Perplexity: 43.7890Shape of captions
torch.Size([64, 13, 256])
Shape of inputs
torch.Size([64, 14, 256])
Epoch [1/3], Step [170/6471], Loss: 3.6079, Perplexity: 36.8877Shape of captions
torch.Size([64, 9, 256])
Shape of inputs
torch.Size([64, 10, 256])
Epoch [1/3], Step [171/6471], Loss: 4.0932, Perplexity: 59.9290Shape of captions
torch.Size([64, 11, 256])
Shape of inputs
torch.Size([64, 12, 256])
Epoch [1/3], Step [172/6471], Loss: 3.5518, Perplexity: 34.8771Shape of captions
torch.Size([64, 10, 256])


Epoch [1/3], Step [221/6471], Loss: 3.4068, Perplexity: 30.1682Shape of captions
torch.Size([64, 14, 256])
Shape of inputs
torch.Size([64, 15, 256])
Epoch [1/3], Step [222/6471], Loss: 3.5632, Perplexity: 35.2761Shape of captions
torch.Size([64, 10, 256])
Shape of inputs
torch.Size([64, 11, 256])
Epoch [1/3], Step [223/6471], Loss: 3.4687, Perplexity: 32.0940Shape of captions
torch.Size([64, 10, 256])
Shape of inputs
torch.Size([64, 11, 256])
Epoch [1/3], Step [224/6471], Loss: 3.6767, Perplexity: 39.5173Shape of captions
torch.Size([64, 12, 256])
Shape of inputs
torch.Size([64, 13, 256])
Epoch [1/3], Step [225/6471], Loss: 3.5625, Perplexity: 35.2503Shape of captions
torch.Size([64, 10, 256])
Shape of inputs
torch.Size([64, 11, 256])
Epoch [1/3], Step [226/6471], Loss: 3.4524, Perplexity: 31.5752Shape of captions
torch.Size([64, 10, 256])
Shape of inputs
torch.Size([64, 11, 256])
Epoch [1/3], Step [227/6471], Loss: 3.6348, Perplexity: 37.8959Shape of captions
torch.Size([64, 10, 256])

...

Epoch [1/3], Step [2696/6471], Loss: 2.2720, Perplexity: 9.6992Shape of captions
torch.Size([64, 12, 256])
Shape of inputs
torch.Size([64, 13, 256])
Epoch [1/3], Step [2697/6471], Loss: 2.2093, Perplexity: 9.1095Shape of captions
torch.Size([64, 11, 256])
Shape of inputs
torch.Size([64, 12, 256])
Epoch [1/3], Step [2698/6471], Loss: 2.4059, Perplexity: 11.0884Shape of captions
torch.Size([64, 14, 256])
Shape of inputs
torch.Size([64, 15, 256])
Epoch [1/3], Step [2699/6471], Loss: 2.5061, Perplexity: 12.2567Shape of captions
torch.Size([64, 11, 256])
Shape of inputs
torch.Size([64, 12, 256])
Epoch [1/3], Step [2700/6471], Loss: 2.0975, Perplexity: 8.1457
Shape of captions
torch.Size([64, 10, 256])
Shape of inputs
torch.Size([64, 11, 256])
Epoch [1/3], Step [2701/6471], Loss: 2.1947, Perplexity: 8.9769Shape of captions
torch.Size([64, 10, 256])
Shape of inputs
torch.Size([64, 11, 256])
Epoch [1/3], Step [2702/6471], Loss: 2.3412, Perplexity: 10.3934Shape of captions
torch.Size([64, 10, 2

Epoch [1/3], Step [2751/6471], Loss: 2.2683, Perplexity: 9.6634Shape of captions
torch.Size([64, 13, 256])
Shape of inputs
torch.Size([64, 14, 256])
Epoch [1/3], Step [2752/6471], Loss: 2.2740, Perplexity: 9.7180Shape of captions
torch.Size([64, 11, 256])
Shape of inputs
torch.Size([64, 12, 256])
Epoch [1/3], Step [2753/6471], Loss: 2.3406, Perplexity: 10.3873Shape of captions
torch.Size([64, 13, 256])
Shape of inputs
torch.Size([64, 14, 256])
Epoch [1/3], Step [2754/6471], Loss: 2.4581, Perplexity: 11.6827Shape of captions
torch.Size([64, 11, 256])
Shape of inputs
torch.Size([64, 12, 256])
Epoch [1/3], Step [2755/6471], Loss: 2.2886, Perplexity: 9.8608Shape of captions
torch.Size([64, 11, 256])
Shape of inputs
torch.Size([64, 12, 256])
Epoch [1/3], Step [2756/6471], Loss: 2.4865, Perplexity: 12.0193Shape of captions
torch.Size([64, 10, 256])
Shape of inputs
torch.Size([64, 11, 256])
Epoch [1/3], Step [2757/6471], Loss: 2.3375, Perplexity: 10.3550Shape of captions
torch.Size([64, 11, 2

Epoch [1/3], Step [2806/6471], Loss: 2.4028, Perplexity: 11.0539Shape of captions
torch.Size([64, 9, 256])
Shape of inputs
torch.Size([64, 10, 256])
Epoch [1/3], Step [2807/6471], Loss: 2.6531, Perplexity: 14.1981Shape of captions
torch.Size([64, 11, 256])
Shape of inputs
torch.Size([64, 12, 256])
Epoch [1/3], Step [2808/6471], Loss: 2.2068, Perplexity: 9.0868Shape of captions
torch.Size([64, 13, 256])
Shape of inputs
torch.Size([64, 14, 256])
Epoch [1/3], Step [2809/6471], Loss: 2.4885, Perplexity: 12.0431Shape of captions
torch.Size([64, 12, 256])
Shape of inputs
torch.Size([64, 13, 256])
Epoch [1/3], Step [2810/6471], Loss: 2.2774, Perplexity: 9.7516Shape of captions
torch.Size([64, 15, 256])
Shape of inputs
torch.Size([64, 16, 256])
Epoch [1/3], Step [2811/6471], Loss: 2.5810, Perplexity: 13.2100Shape of captions
torch.Size([64, 12, 256])
Shape of inputs
torch.Size([64, 13, 256])
Epoch [1/3], Step [2812/6471], Loss: 2.2750, Perplexity: 9.7283Shape of captions
torch.Size([64, 11, 25

Epoch [1/3], Step [2861/6471], Loss: 2.1768, Perplexity: 8.8181Shape of captions
torch.Size([64, 31, 256])
Shape of inputs
torch.Size([64, 32, 256])
Epoch [1/3], Step [2862/6471], Loss: 4.0032, Perplexity: 54.7752Shape of captions
torch.Size([64, 12, 256])
Shape of inputs
torch.Size([64, 13, 256])
Epoch [1/3], Step [2863/6471], Loss: 2.5214, Perplexity: 12.4462Shape of captions
torch.Size([64, 21, 256])
Shape of inputs
torch.Size([64, 22, 256])
Epoch [1/3], Step [2864/6471], Loss: 3.1152, Perplexity: 22.5379Shape of captions
torch.Size([64, 13, 256])
Shape of inputs
torch.Size([64, 14, 256])
Epoch [1/3], Step [2865/6471], Loss: 2.3185, Perplexity: 10.1606Shape of captions
torch.Size([64, 12, 256])
Shape of inputs
torch.Size([64, 13, 256])
Epoch [1/3], Step [2866/6471], Loss: 2.1599, Perplexity: 8.6703Shape of captions
torch.Size([64, 10, 256])
Shape of inputs
torch.Size([64, 11, 256])
Epoch [1/3], Step [2867/6471], Loss: 2.3552, Perplexity: 10.5402Shape of captions
torch.Size([64, 11, 

Epoch [1/3], Step [2916/6471], Loss: 2.2321, Perplexity: 9.3192Shape of captions
torch.Size([64, 13, 256])
Shape of inputs
torch.Size([64, 14, 256])
Epoch [1/3], Step [2917/6471], Loss: 2.4252, Perplexity: 11.3050Shape of captions
torch.Size([64, 12, 256])
Shape of inputs
torch.Size([64, 13, 256])
Epoch [1/3], Step [2918/6471], Loss: 2.1903, Perplexity: 8.9377Shape of captions
torch.Size([64, 11, 256])
Shape of inputs
torch.Size([64, 12, 256])
Epoch [1/3], Step [2919/6471], Loss: 2.4243, Perplexity: 11.2943Shape of captions
torch.Size([64, 10, 256])
Shape of inputs
torch.Size([64, 11, 256])
Epoch [1/3], Step [2920/6471], Loss: 2.4338, Perplexity: 11.4022Shape of captions
torch.Size([64, 12, 256])
Shape of inputs
torch.Size([64, 13, 256])
Epoch [1/3], Step [2921/6471], Loss: 2.4565, Perplexity: 11.6641Shape of captions
torch.Size([64, 13, 256])
Shape of inputs
torch.Size([64, 14, 256])
Epoch [1/3], Step [2922/6471], Loss: 2.3740, Perplexity: 10.7407Shape of captions
torch.Size([64, 11, 

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [ ]:
# (Optional) TODO: Validate your model.